In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse
from scipy import stats
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

import time

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import folium
import fbprophet
plt.style.use('fivethirtyeight')
#import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

D:\AnacondaIDE\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import itertools
from datetime import timedelta

#from joblib import Parallel, delayed
#import multiprocessing

from sklearn.preprocessing import MinMaxScaler

import tensorflow
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.models import load_model
#from keras.datasets import imdb

D:\AnacondaIDE\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
train_16 = pd.read_csv('D:/ML work/Zillow Home Value Prediction/train_2016_v2/train_2016_v2.csv')
properties_16 = pd.read_csv('D:/ML work/Zillow Home Value Prediction/properties_2016/properties_2016.csv')

train_17 = pd.read_csv('D:/ML work/Zillow Home Value Prediction/train_2017.csv/train_2017.csv')
properties_17 = pd.read_csv('D:/ML work/Zillow Home Value Prediction/properties_2017.csv/properties_2017.csv')

In [4]:
train_16['transactiondate'] = pd.to_datetime(train_16['transactiondate'], format = '%Y-%m-%d')
train_17['transactiondate'] = pd.to_datetime(train_17['transactiondate'], format = '%Y-%m-%d')

In [5]:
properties_df_16 = properties_16.merge(train_16, how = 'left', on = 'parcelid')
del properties_16, train_16

properties_df_17 = properties_17.merge(train_17, how = 'left', on = 'parcelid')
del properties_17, train_17

In [6]:
properties_df = properties_df_16

In [7]:
del properties_df_16#, properties_df_17
gc.collect()

42

In [8]:
properties_df.drop(['taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount'], axis = 1, inplace = True)

In [9]:
for c, dtype in zip(properties_df.columns, properties_df.dtypes):
    if dtype == np.float64:
        properties_df[c] = properties_df[c].astype(np.float32) 
    elif dtype == np.int64:
        properties_df[c] = properties_df[c].astype(np.int32)
#end for loop

gc.collect()

7

In [10]:
error_mean = np.mean(properties_df['logerror'])
error_median = np.median(properties_df['logerror'].dropna())
error_std = np.std(properties_df['logerror'])

error_mean, error_median, error_std

(0.01145784743130207, 0.0060000001, 0.16107094287872314)

In [11]:
properties_df['calculatedbathnbr'] = np.where(np.isnan(properties_df['calculatedbathnbr']), properties_df['bathroomcnt'], properties_df['calculatedbathnbr'])

In [12]:
#drop bathroomcnt
properties_df.drop('bathroomcnt', axis = 1, inplace = True)

In [13]:
# So we can safely drop one column of them
properties_df.drop('finishedsquarefeet50', axis = 1, inplace = True)

In [14]:
# if condition is satisfied then replace with True else False
# assume that if fireplacecnt is null, then fireplaceflg will be false
properties_df['fireplaceflag'] = np.where(pd.isnull(properties_df['fireplaceflag']) & pd.notnull(properties_df['fireplacecnt']) & properties_df['fireplacecnt'] > 0, True, False)

In [15]:
properties_df['fireplaceflag'].count()
# fireplaceflag is recovered

2985342

In [16]:
index = properties_df['fireplacecnt'].isnull()
properties_df.loc[index, 'fireplacecnt'] = 0

In [17]:
properties_df['house_age'] = datetime.now().date().year - properties_df['yearbuilt']

In [18]:
#assume if taxdelinquencyflag is null then False (doesn't exist)
index = properties_df['taxdelinquencyflag'].isnull()
properties_df.loc[index, 'taxdelinquencyflag'] = False

In [19]:
properties_df.drop('pooltypeid10', axis = 1, inplace = True)

In [20]:
# this does not make sense in theory. it should be assigned poolcnt - pooltypeid7
index = (properties_df['pooltypeid2'].isnull()) & (properties_df['hashottuborspa'] == True) & (properties_df['poolcnt'].notnull()) & ((properties_df['pooltypeid7']).notnull())
properties_df.loc[index, 'pooltypeid2'] = properties_df.loc[index, 'poolcnt'] - properties_df.loc[index, 'pooltypeid7']

In [21]:
index = (properties_df['pooltypeid2'].isnull()) & (properties_df['hashottuborspa'] == True) & (properties_df['poolcnt'].notnull()) & ((properties_df['pooltypeid7']).isnull())
properties_df.loc[index, 'pooltypeid2'] = properties_df['poolcnt']

index = (properties_df['pooltypeid2'].isnull()) & (properties_df['poolcnt'].isnull())
properties_df.loc[index, 'pooltypeid2'] = 0.0

In [22]:
#this does not make sense, assign poolcnt to pooltypeid7
index = (properties_df['pooltypeid7'].isnull()) & (properties_df['poolcnt'].notnull() & (properties_df['pooltypeid2'].notnull())) 
properties_df.loc[index, 'pooltypeid7'] = properties_df.loc[index, 'poolcnt'] - properties_df.loc[index, 'pooltypeid2']
 

In [23]:
index = properties_df.hashottuborspa.isnull()
properties_df.loc[index, 'hashottuborspa'] = False

index = properties_df.pooltypeid2.isnull()
properties_df.loc[index,'pooltypeid2'] = 0

index = properties_df.pooltypeid7.isnull()
properties_df.loc[index,'pooltypeid7'] = 0

index = properties_df.poolcnt.isnull()
properties_df.loc[index,'poolcnt'] = 0

index = (properties_df.poolcnt == 0)
properties_df.loc[index, 'poolsizesum'] = 0

In [24]:
# replace 'Y' with True in taxdelinquencyflag
index = (properties_df.taxdelinquencyflag == 'Y')
properties_df.loc[index,'taxdelinquencyflag'] = True

In [25]:
properties_df['taxdelinquencyflag'] = properties_df.taxdelinquencyflag.astype(np.bool)
properties_df['hashottuborspa'] = properties_df.hashottuborspa.astype(np.bool)

In [26]:
#Null in garage count means there are no garages
index = properties_df.garagecarcnt.isnull()
properties_df.loc[index,'garagecarcnt'] = 0

In [27]:
#error in calculation of the finished living area of home
properties_df['N-LivingAreaError'] = properties_df['calculatedfinishedsquarefeet'] / properties_df['finishedsquarefeet12']


In [28]:
#proportion of living area
properties_df['N-LivingAreaProp'] = properties_df['calculatedfinishedsquarefeet'] / properties_df['lotsizesquarefeet']
properties_df['N-LivingAreaProp2'] = properties_df['finishedsquarefeet12'] / properties_df['finishedsquarefeet15']

#Amout of extra space
properties_df['N-ExtraSpace'] = properties_df['lotsizesquarefeet'] - properties_df['calculatedfinishedsquarefeet'] 
properties_df['N-ExtraSpace-2'] = properties_df['finishedsquarefeet15'] - properties_df['finishedsquarefeet12']

#Average room size
properties_df['N-AvRoomSize'] = properties_df['calculatedfinishedsquarefeet'] / properties_df['roomcnt']

#Total number of rooms
properties_df['N-TotalRooms'] = properties_df['calculatedbathnbr'] + properties_df['bedroomcnt']

# Number of Extra rooms
properties_df['N-ExtraRooms'] = properties_df['roomcnt'] - properties_df['N-TotalRooms']

#Does property have a garage, pool or hot tub and AC?
properties_df['N-GarPoolAC'] = ((properties_df['garagecarcnt']>0) & (properties_df['hashottuborspa']>0) & (properties_df['airconditioningtypeid']!=5))*1

properties_df["N-location"] = properties_df["latitude"] + properties_df["longitude"]
properties_df["N-location-2"] = properties_df["latitude"] * properties_df["longitude"]
properties_df["N-location-2round"] = properties_df["N-location-2"].round(-4)

properties_df["N-latitude-round"] = properties_df["latitude"].round(-4)
properties_df["N-longitude-round"] = properties_df["longitude"].round(-4)

# #Ratio of tax of property over parcel
# properties_df['N-ValueRatio'] = properties_df['taxvaluedollarcnt'] / properties_df['taxamount']

# #TotalTaxScore
# properties_df['N-TaxScore'] = properties_df['taxvaluedollarcnt']*properties_df['taxamount']

#polnomials of tax delinquency year
properties_df["N-taxdelinquencyyear-2"] = properties_df["taxdelinquencyyear"] ** 2
properties_df["N-taxdelinquencyyear-3"] = properties_df["taxdelinquencyyear"] ** 3

#Length of time since unpaid taxes
properties_df['N-life'] = 2018 - properties_df['taxdelinquencyyear']

#Number of properties in the zip
zip_count = properties_df['regionidzip'].value_counts().to_dict()
properties_df['N-zip_count'] = properties_df['regionidzip'].map(zip_count)

#Number of properties in the city
city_count = properties_df['regionidcity'].value_counts().to_dict()
properties_df['N-city_count'] = properties_df['regionidcity'].map(city_count)

#Number of properties in the city
region_count = properties_df['regionidcounty'].value_counts().to_dict()
properties_df['N-county_count'] = properties_df['regionidcounty'].map(city_count)

#Indicator whether it has AC or not
properties_df['N-ACInd'] = (properties_df['airconditioningtypeid']!=5)*1

#Indicator whether it has Heating or not 
properties_df['N-HeatInd'] = (properties_df['heatingorsystemtypeid']!=13)*1

#polnomials of the variable
properties_df["N-structuretaxvaluedollarcnt-2"] = properties_df["structuretaxvaluedollarcnt"] ** 2
properties_df["N-structuretaxvaluedollarcnt-3"] = properties_df["structuretaxvaluedollarcnt"] ** 3

#Average structuretaxvaluedollarcnt by city
group = properties_df.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
properties_df['N-Avg-structuretaxvaluedollarcnt'] = properties_df['regionidcity'].map(group)

#Deviation away from average
properties_df['N-Dev-structuretaxvaluedollarcnt'] = abs((properties_df['structuretaxvaluedollarcnt'] - properties_df['N-Avg-structuretaxvaluedollarcnt']))/properties_df['N-Avg-structuretaxvaluedollarcnt']

properties_df['hashottuborspa'] = properties_df['hashottuborspa'].astype(np.int32)
properties_df['fireplaceflag'] = properties_df['fireplaceflag'].astype(np.int32)
properties_df['taxdelinquencyflag'] = properties_df['taxdelinquencyflag'].astype(np.int32)
properties_df['fireplaceflag'] = properties_df['fireplaceflag'].astype(np.int32)

In [29]:
properties_df['airconditioningtypeid'].fillna(value=0.0, inplace = True)
properties_df['architecturalstyletypeid'].fillna(value=0.0, inplace = True)
properties_df['basementsqft'].fillna(value=0.0, inplace = True)
properties_df['bedroomcnt'].fillna(value=0.0, inplace = True)
properties_df['buildingclasstypeid'].fillna(value=0.0, inplace = True)
properties_df['buildingqualitytypeid'].fillna(value=0.0, inplace = True)

properties_df['decktypeid'].fillna(value=0.0, inplace = True)
properties_df['finishedfloor1squarefeet'].fillna(value=0.0, inplace = True)

properties_df['finishedsquarefeet12'].fillna(value=0.0, inplace = True)
properties_df['finishedsquarefeet13'].fillna(value=0.0, inplace = True)
properties_df['finishedsquarefeet15'].fillna(value=0.0, inplace = True)
properties_df['finishedsquarefeet6'].fillna(value=0.0, inplace = True)
properties_df['fips'].fillna(value=0.0, inplace = True)
properties_df['censustractandblock'].fillna(value = 0.0, inplace = True)
properties_df['taxdelinquencyyear'].fillna(value = 0.0, inplace = True)
properties_df['heatingorsystemtypeid'].fillna(value=0.0, inplace = True)
properties_df['latitude'].fillna(value=0.0, inplace = True)
properties_df['longitude'].fillna(value=0.0, inplace = True)
properties_df['propertylandusetypeid'].fillna(value = 0.0, inplace = True)
properties_df['rawcensustractandblock'].fillna(value = 0.0, inplace = True)

properties_df['regionidcity'].fillna(value = 0.0, inplace = True)
properties_df['regionidcounty'].fillna(value = 0.0, inplace = True)
properties_df['regionidneighborhood'].fillna(value = 0.0, inplace = True)
properties_df['regionidzip'].fillna(value = 0.0, inplace = True)
properties_df['roomcnt'].fillna(value = 0.0, inplace = True)
properties_df['yearbuilt'].fillna(value = 0.0, inplace = True)
properties_df['structuretaxvaluedollarcnt'].fillna(value = 0.0, inplace = True)
# properties_df['taxvaluedollarcnt'].fillna(value = 0.0, inplace = True)
properties_df['assessmentyear'].fillna(value = 0.0, inplace = True)
# properties_df['landtaxvaluedollarcnt'].fillna(value = 0.0, inplace = True)
# properties_df['taxamount'].fillna(value = 0.0, inplace = True)
properties_df['numberofstories'].fillna(value = 0.0, inplace = True)

properties_df['poolsizesum'].fillna(value=0.0, inplace = True)
properties_df['house_age'].fillna(value=1.0, inplace = True)
properties_df['N-LivingAreaError'].fillna(value=0.0, inplace = True)
properties_df['N-LivingAreaProp'].fillna(value=0.0, inplace = True)
properties_df['N-ExtraSpace'].fillna(value=0.0, inplace = True)
properties_df['N-TotalRooms'].fillna(value=0.0, inplace = True)
properties_df['N-ExtraRooms'].fillna(value=0.0, inplace = True)
# properties_df['N-ValueProp'].fillna(value=0.0, inplace = True)
properties_df['N-location'].fillna(value=0.0, inplace = True)
properties_df['N-location-2'].fillna(value=0.0, inplace = True)
properties_df['N-location-2round'].fillna(value=0.0, inplace = True)
properties_df['N-latitude-round'].fillna(value=0.0, inplace = True)
properties_df['N-longitude-round'].fillna(value=0.0, inplace = True)
# properties_df['N-ValueRatio'].fillna(value=0.0, inplace = True)
# properties_df['N-TaxScore'].fillna(value=0.0, inplace = True)
properties_df['N-taxdelinquencyyear-2'].fillna(value=0.0, inplace = True)
properties_df['N-taxdelinquencyyear-3'].fillna(value=0.0, inplace = True)
properties_df['N-life'].fillna(value=0.0, inplace = True)
properties_df['N-zip_count'].fillna(value=0.0, inplace = True)
properties_df['N-city_count'].fillna(value=0.0, inplace = True)
properties_df['N-structuretaxvaluedollarcnt-2'].fillna(value=0.0, inplace = True)
properties_df['N-structuretaxvaluedollarcnt-3'].fillna(value=0.0, inplace = True)
properties_df['N-Avg-structuretaxvaluedollarcnt'].fillna(value=0.0, inplace = True)
properties_df['N-Dev-structuretaxvaluedollarcnt'].fillna(value=0.0, inplace = True)


In [30]:
properties_df['storytypeid'].fillna(value=0.0, inplace = True)

properties_df['threequarterbathnbr'].fillna(value=0.0, inplace = True)

properties_df['typeconstructiontypeid'].fillna(value=0.0, inplace = True)

properties_df['unitcnt'].fillna(value=1.0, inplace = True)

properties_df['calculatedfinishedsquarefeet'].fillna(value=0.0,inplace = True)
properties_df['yardbuildingsqft17'].fillna(value=0.0,inplace = True)
properties_df['yardbuildingsqft26'].fillna(value=0.0,inplace = True)
properties_df['calculatedbathnbr'].fillna(value=0.0, inplace = True)

properties_df['fullbathcnt'].fillna(value=0.0, inplace = True)

properties_df['garagetotalsqft'].fillna(value=0.0, inplace = True)

properties_df['lotsizesquarefeet'].fillna(value=0.0, inplace = True)
properties_df.drop(['N-county_count', 'N-ExtraSpace-2', 'N-LivingAreaProp2'], inplace = True, axis = 1)
#because this column contains so much inf values
properties_df.drop(['N-AvRoomSize'], axis = 1, inplace = True)

In [31]:
properties_df.drop_duplicates(subset = ['parcelid'], inplace = True, keep = 'last')

In [32]:
for c, dtype in zip(properties_df.columns, properties_df.dtypes):
    if (dtype == np.float64) | (dtype == np.int):
        properties_df[c] = properties_df[c].astype(np.float32) 
#     elif dtype == np.int64:
#         properties_df[c] = properties_df[c].astype(np.int32)
#end for loop

gc.collect()

83

In [33]:
scaler = MinMaxScaler(feature_range=(-1, 1))



In [34]:
exog_train = properties_df[properties_df['logerror'].notnull()].drop(['logerror', 'parcelid', 'propertyzoningdesc', 'propertycountylandusecode'], axis = 1)

exog_test = properties_df[properties_df['logerror'].isnull()]

parcelids_test = exog_test['parcelid']

exog_test.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis = 1, inplace = True)

timeseries_train = properties_df[properties_df['logerror'].notnull()][['transactiondate', 'logerror']]

timeseries_test = properties_df[properties_df['logerror'].isnull()][['transactiondate', 'logerror']]


In [35]:
exog_train.index = exog_train['transactiondate']
exog_train.drop(['transactiondate'], axis = 1, inplace = True)

timeseries_train.index = timeseries_train['transactiondate']
timeseries_train.drop(['transactiondate'], axis = 1, inplace = True)

timeseries_test.index = timeseries_test['transactiondate']
timeseries_test.drop(['transactiondate'], axis = 1, inplace = True)

In [36]:
timeseries_train_grouped = timeseries_train.groupby('transactiondate').mean()
exog_train_grouped = exog_train.groupby('transactiondate').mean()

In [37]:
exog_train_grouped = exog_train_grouped.asfreq('d', method = 'bfill')
timeseries_train_grouped = timeseries_train_grouped.asfreq('d', method = 'bfill')

In [38]:
#exog_train_grouped.loc[ : pd.to_datetime('2016-09-30')]

In [39]:
scaler_exog = scaler.fit(exog_train_grouped)
scaler_endog = scaler.fit(timeseries_train_grouped)

#transformed_exog_train = scaler_exog.transform(exog_train_grouped)
#transformed_endog_train = scaler_endog.transform(timeseries_train_grouped)

In [40]:
endog_sep16 = timeseries_train_grouped.loc[ : pd.to_datetime('2016-09-30')]
endog_oct16 = timeseries_train_grouped.loc[ : pd.to_datetime('2016-10-31')]
endog_nov16 = timeseries_train_grouped.loc[ : pd.to_datetime('2016-11-30')]
# endog_17 = timeseries_train_grouped.iloc[ : len(timeseries_train_grouped)]

exog_sep16 = exog_train_grouped.loc[ : pd.to_datetime('2016-09-30')]
exog_oct16 = exog_train_grouped.loc[ : pd.to_datetime('2016-10-31')]
exog_nov16 = exog_train_grouped.loc[ : pd.to_datetime('2016-11-30')]
# exog_17 = exog_train_grouped.iloc[ : len(exog_train_grouped)]


# make record of start and end dates of endog before reshaping (after reshaping we will lose time index)
endog_sep16_start = endog_sep16.iloc[0].name
endog_sep16_end = endog_sep16.iloc[len(endog_sep16) - 1].name

endog_oct16_start = endog_oct16.iloc[0].name
endog_oct16_end = endog_oct16.iloc[len(endog_oct16) - 1].name

endog_nov16_start = endog_nov16.iloc[0].name
endog_nov16_end = endog_nov16.iloc[len(endog_nov16) - 1].name

# endog_17_start = endog_17.iloc[0].name
# endog_17_end = endog_17.iloc[len(endog_17) - 1].name


# transform the data to MinMaxScaler
endog_sep16 = scaler_endog.transform(endog_sep16)
endog_oct16 = scaler_endog.transform(endog_oct16)
endog_nov16 = scaler_endog.transform(endog_nov16)
# endog_17 = scaler_endog.transfrom(endog_17)

exog_sep16 = scaler_exog.transform(exog_sep16)
exog_oct16 = scaler_exog.transform(exog_oct16)
exog_nov16 = scaler_exog.transform(exog_nov16)
# exog_17 = scaler_exog.transform(exog_17)

exog_test = scaler_exog.transform(exog_test)


In [107]:
# exog_sep16 = exog_sep16[:6,]

In [41]:
exog2_sep16 = np.array([])
exog2_sep16 = np.concatenate([exog_sep16[x : x + 30,:].reshape(1, 30, exog_sep16.shape[1]) for x in range(exog_sep16.shape[0]-29)])

endog_sep16 = endog_sep16.reshape(endog_sep16.shape[0],)
endog_sep16 = endog_sep16[ 29 : ]

In [42]:
exog2_sep16.shape, endog_sep16.shape

((245, 30, 72), (245,))

In [82]:
# examples = 60 # how far back to look
# y_examples = 92 # how many steps forward to predict
# nb_samples = len(exog_train_grouped) - examples - y_examples

# input_list = [np.expand_dims(np.atleast_2d(exog_sep16[i:examples+i,:]), axis=0) for i in range(nb_samples)]
# exog2_sep16 = np.concatenate(input_list, axis=0)

# test_input_list = [np.expand_dims(np.atleast_2d(exog_test[len(exog_test)-examples:len(exog_test),:]), axis=0) for i in range(1)]
# exog2_test = np.concatenate(test_input_list, axis=0)


In [87]:
exog2_sep16.shape, exog_test.shape

((245, 30, 72), (2895037, 72))

In [48]:


# reshape enxog variables according to the specifications of keras lstm

# exog_sep16 = exog_sep16.reshape(exog_sep16.shape[0], 1, exog_sep16.shape[1])
# exog_oct16 = exog_oct16.reshape(exog_oct16.shape[0], 1, exog_oct16.shape[1])
# exog_nov16 = exog_nov16.reshape(exog_nov16.shape[0], 1, exog_nov16.shape[1])
# # exog_17 = exog_17.reshape(exog_17.shape[0], 1, exog_17.shape[1])

# endog_sep16 = endog_sep16.reshape(endog_sep16.shape[0],)
# endog_oct16 = endog_oct16.reshape(endog_oct16.shape[0],)
# endog_nov16 = endog_nov16.reshape(endog_nov16.shape[0],)
# # endog_17 = endog_17.reshape(endog_17.shape[0], )

exog_test = exog_test.reshape(exog_test.shape[0], 1, exog_test.shape[1])

In [43]:
#endog_exog_tuples = [(endog_sep16, exog_sep16), (endog_oct16, exog_oct16), (endog_nov16, exog_nov16)]
endog_exog_tuples = [(endog_sep16, exog2_sep16, endog_sep16_start, endog_sep16_end)
#                     (endog_oct16, exog_oct16, endog_oct16_start, endog_oct16_end),
#                     (endog_nov16, exog_nov16, endog_nov16_start, endog_nov16_end)
#                     (endog_17, exog_17, endog_17_start, endog_17_end)
                    ]

In [1189]:
from joblib import Parallel, delayed
import multiprocessing

In [1190]:
# predictions = {'201610' : pd.DataFrame(columns = ['201610'], dtype = np.float32),
#                 '201611' : pd.DataFrame(columns = ['201611'], dtype = np.float32),
#                 '201612' : pd.DataFrame(columns = ['201612'], dtype = np.float32),
#                 '2017' : pd.DataFrame(columns = ['2017'], dtype = np.float32),
#                 #'201711' : pd.DataFrame(columns = ['201711'], dtype = np.float32),
#                 #'201712' : pd.DataFrame(columns = ['201712'], dtype = np.float32)
#                }


# def get_predictions(rows, prediction_column):
#     global predictions
    
#     #prediction = sarimax_result.get_prediction(start = start, end = end, dynamic = True, full_results = True, exog = rows)
    
#     model = load_model('model_' + prediction_column + '.h5')
#     # make predictions
#     prediction = model.predict(rows, verbose=0)
#     print(prediction.shape)
    
#     df = pd.DataFrame(prediction[len(prediction) - 1], columns = [prediction_column])
    
#     predictions[prediction_column] = predictions[prediction_column].append(df)

# #get_predictions end


# for endog, exog, endog_start, endog_end in endog_exog_tuples:
# # def sarimax_model_func(endog, exog):
# #     sarimax_model = SARIMAX(endog = endog,
# #                                     order = (0, 0, 0),
# #                                     exog = exog,
# #                                     seasonal_order=(1, 0, 0, 12),
# #                                     enforce_stationarity=False,
# #                                     enforce_invertibility=False)

# #     sarimax_result = sarimax_model.fit()

#     model = Sequential()
#     model.add(LSTM(100, return_sequences=True, input_shape = (exog.shape[1], exog.shape[2])))
#     model.add(LSTM(100))
#     model.add(Dense(1, activation='linear'))
    
#     model.compile(loss='mse', optimizer='adam')
    
#     model.fit(exog, endog, epochs=500, shuffle=False, verbose=1)
    
#     start_date = endog_end + timedelta(days = 1)
#     end_date = endog_end + timedelta(days = 15)
    
#     delta = end_date - start_date
    
#     year_of_prediction = end_date.year
#     month_of_prediction = end_date.month

#     prediction_column = str(year_of_prediction) + str(month_of_prediction)
    
#     model.save('model_' + prediction_column + '.h5')
    
#     if (str(year_of_prediction) == '2017'):
#         prediction_column = str(year_of_prediction)
    
    
#     for x in range(0, len(exog_test)):
#         record = np.array([exog_test[x]])
        
# #         record = record.append([record] * (delta.days - 1))
# #         record.index = range(0, delta.days)
#         if (prediction_column != '2017'):
#             record = np.repeat(record, delta.days + 1, axis = 0)
            
#         else:
#             record = np.repeat(record, 92, axis = 0)      #92 is number of days of October + November + December
    
#         record = record.reshape(record.shape[0], 1, record.shape[1])

# #         datelist = pd.date_range(start_date, periods = delta.days, freq = 'D')
# #         transactiondate = pd.DataFrame(datelist, columns = ['transactiondate'])

# #         record.index = datelist
# #         record.index.name = 'transactiondate'
        
#         prediction = get_predictions(record, prediction_column)

In [45]:
predictions = {'201610' : pd.DataFrame(columns = ['201610'], dtype = np.float32),
                '201611' : pd.DataFrame(columns = ['201611'], dtype = np.float32),
                '201612' : pd.DataFrame(columns = ['201612'], dtype = np.float32),
                '2017' : pd.DataFrame(columns = ['2017'], dtype = np.float32),
                #'201711' : pd.DataFrame(columns = ['201711'], dtype = np.float32),
                #'201712' : pd.DataFrame(columns = ['201712'], dtype = np.float32)
               }


def get_predictions(rows, prediction_column):
    global predictions
    
    #prediction = sarimax_result.get_prediction(start = start, end = end, dynamic = True, full_results = True, exog = rows)
    
    model = load_model('model_' + prediction_column + '.h5')
    # make predictions
    prediction = model.predict(rows, verbose=0)
    print(prediction.shape)
    
    df = pd.DataFrame(prediction[len(prediction) - 1], columns = [prediction_column])
    
    predictions[prediction_column] = predictions[prediction_column].append(df)

#get_predictions end


for endog, exog, endog_start, endog_end in endog_exog_tuples:
# def sarimax_model_func(endog, exog):
#     sarimax_model = SARIMAX(endog = endog,
#                                     order = (0, 0, 0),
#                                     exog = exog,
#                                     seasonal_order=(1, 0, 0, 12),
#                                     enforce_stationarity=False,
#                                     enforce_invertibility=False)

#     sarimax_result = sarimax_model.fit()

    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape = (exog.shape[1], exog.shape[2])))
    model.add(LSTM(100, return_sequences = False))
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mse', optimizer='adam')
    
    model.fit(exog, endog, epochs=50, shuffle=False, verbose=1)
    
    start_date = endog_end + timedelta(days = 1)
    end_date = endog_end + timedelta(days = 15)
    
    delta = end_date - start_date
    
    year_of_prediction = end_date.year
    month_of_prediction = end_date.month

    prediction_column = str(year_of_prediction) + str(month_of_prediction)
    
    model.save('model_lstm.h5')
    
    if (str(year_of_prediction) == '2017'):
        prediction_column = str(year_of_prediction)
    
    

Epoch 1/50
245/245 [==============================] - 4s - loss: 0.0525     
Epoch 2/50
245/245 [==============================] - 0s - loss: 0.0349     
Epoch 3/50
245/245 [==============================] - 0s - loss: 0.0375     
Epoch 4/50
245/245 [==============================] - 0s - loss: 0.0249     
Epoch 5/50
245/245 [==============================] - 0s - loss: 0.0251     
Epoch 6/50
245/245 [==============================] - 0s - loss: 0.0216     
Epoch 7/50
245/245 [==============================] - 0s - loss: 0.0224     
Epoch 8/50
245/245 [==============================] - 0s - loss: 0.0215     
Epoch 9/50
245/245 [==============================] - 0s - loss: 0.0210     
Epoch 10/50
245/245 [==============================] - 0s - loss: 0.0212     
Epoch 11/50
245/245 [==============================] - 0s - loss: 0.0207     
Epoch 12/50
245/245 [==============================] - 0s - loss: 0.0205     
Epoch 13/50
245/245 [==============================] - 0s - loss: 0.0204 

In [52]:
model = load_model('model_lstm.h5')

prediction = model.predict(exog_test, verbose=1)
#     for x in range(0, len(exog_test)):

ValueError: Error when checking : expected lstm_3_input to have shape (None, 30, 72) but got array with shape (2895067, 1, 72)

In [47]:
model.output_shape

(None, 1)

In [46]:
model.input_shape

(None, 30, 72)

In [868]:
prediction

NameError: name 'prediction' is not defined

In [1205]:
endog_sep16


array([-0.7238661 , -0.43128777, -0.31542821, -0.35185375, -0.26344065,
       -0.25590285, -0.34791175, -0.28571553, -0.13263179, -0.31547805,
       -0.26223067, -0.34433794, -0.15443062, -0.22619402, -0.20622291,
        0.13502814, -0.20742747, -0.36874538, -0.30429553, -0.20858801,
       -0.32533195, -0.21030938, -0.24567617, -0.34174079, -0.09912329,
       -0.11859562, -0.24164009, -0.20844921, -0.30892482, -0.30778981,
       -0.37169636, -0.22513671, -0.2436683 , -0.1569894 , -0.09575444,
       -0.21681482, -0.396791  , -0.17618252, -0.29318558, -0.17295309,
       -0.18015012, -0.30362712, -0.22839623, -0.3404876 , -0.75625998,
       -0.55692539, -0.32454138, -0.21400226, -0.21648424, -0.24459758,
       -0.22136556, -0.41754835, -0.2125102 , -0.34707138, -0.30005142,
       -0.28744029, -0.22827616, -0.42624689, -0.47433832, -0.29534625,
       -0.27684467, -0.33520319, -0.24394325, -0.18392483, -0.51092024,
       -0.40264436, -0.23071884, -0.33889317, -0.29504415, -0.28

In [1206]:
exog_sep16

array([[[  4.49027601e+00,  -3.71848297e-01,  -3.71848297e-01, ...,
           1.06175239e+17,   1.24734085e+06,   5.16668131e+00]],

       [[  2.05921386e+00,  -3.71848297e-01,  -3.71848297e-01, ...,
           8.44426594e+17,   1.39098642e+06,   4.01803817e+00]],

       [[  9.25515792e+00,  -3.71848297e-01,  -3.71848297e-01, ...,
           4.08800518e+17,   1.24441013e+06,   3.69129399e+00]],

       ..., 
       [[  3.80037999e+00,  -3.71848297e-01,  -3.71848297e-01, ...,
           1.35414902e+18,   1.23823291e+06,   3.54601899e+00]],

       [[  3.73994822e+00,  -3.71848297e-01,   8.40698741e+00, ...,
           3.93863169e+17,   1.25693299e+06,   3.29993440e+00]],

       [[  3.56843035e+00,  -3.71848297e-01,   1.53710656e+00, ...,
           3.28306336e+17,   1.22667253e+06,   3.36569960e+00]]])

In [49]:
exog_test

array([[[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
          -3.71848297e-01,   2.73982042e+06,  -3.71848297e-01]],

       [[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
          -3.71848297e-01,   2.73982042e+06,  -3.71848297e-01]],

       [[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
           2.00988794e+18,   1.05992636e+06,   2.49920150e+01]],

       ..., 
       [[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
          -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01]],

       [[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
          -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01]],

       [[ -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01, ...,
          -3.71848297e-01,  -3.71848297e-01,  -3.71848297e-01]]])

In [947]:
exog_sep16.shape

(1, 274, 72)

In [50]:
exog_test.shape

(2895067, 1, 72)

In [946]:
endog_sep16.shape

(1, 274)

In [917]:
len(exog_sep16)

1